In [ ]:
%matplotlib inline
from os.path import basename, splitext
import glob

import numpy as np
import matplotlib.pyplot as plt
import typhon.plots
from netCDF4 import Dataset


plt.style.use(typhon.plots.styles('typhon'))


def extract_scale(scenario):
    return splitext(basename(scenario))[0].split('x')[-1].replace('_', ', ').split('-')[0]


season = 'tropical'

scenarios = glob.iglob('results/double_co2/{season}_co2_x*-re.nc'.format(season=season))

fig, [ax, ax2] = plt.subplots(nrows=2, sharex=True)
for scenario in sorted(scenarios, reverse=True):
    data = Dataset(scenario)
    time = data.variables['time'][:] / 24  # time in days
    
    # Calculate surface temperatures from radiative fluxes.
    lw_flxd = data.variables['lw_flxd'][:, 0]
    sw_flxu = data.variables['sw_flxu'][:, 0]
    sw_flxd = data.variables['sw_flxd'][:, 0]
    Ts = ((sw_flxd - sw_flxu + lw_flxd) / 5.67e-8)**(0.25) 
    
    ax.plot(time, Ts,
            label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
            linewidth=3,
            )
    
    lw_flxu = data.variables['lw_flxu'][:, -1]
    lw_flxd = data.variables['lw_flxd'][:, -1]
    sw_flxu = data.variables['sw_flxu'][:, -1]
    sw_flxd = data.variables['sw_flxd'][:, -1]
    toa = ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))
    
    ax2.plot(time, toa,
             label=r'$\alpha = {}$'.format(extract_scale(scenario)),
             linewidth=3,
             )
    
fig.suptitle('FASCOD {season}'.format(season=season.capitalize()))
ax.grid('on')
ax2.grid('on')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_xlabel('Day')
ax.set_xlim(left=0)
ax.set_ylabel('$T_s$ [K]')
ax2.set_ylabel('TOA [$Wm^{-2}$]')
fig.savefig('plots/t_surface_double_co2_{season}_re.pdf'.format(season=season))

In [ ]:
season = 'tropical'
scenarios = glob.iglob('results/double_co2/{season}_co2_x*-re.nc'.format(season=season))

fig, ax = plt.subplots(figsize=typhon.plots.figsize(6, portrait=True))
for scenario in sorted(scenarios, reverse=True):
    data = Dataset(scenario)
    p = data.variables['plev'][:]
    T = data.variables['T'][-1, :]
    ci = np.argmin(T)
    q = data.variables['H2O'][-1, :]
    
    rh = typhon.atmosphere.relative_humidity(q, p, T)

    l = typhon.plots.profile_p_log(
        p, T,
        label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
        linewidth=3,
        )
    ax.axhline(p[ci], linestyle='dotted', color=l[0].get_color(), alpha=0.3)
    
ax.set_title('FASCOD {season}'.format(season=season.capitalize()))
ax.grid('on')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('Temperature [K]')
ax.set_ylim(top=p.min(), bottom=p.max())
fig.savefig('plots/t_profile_double_co2_{season}_re.pdf'.format(season=season))